# GAN

## 用語解説

GAN Generative Adversarial Network 敵対的生成ネットワーク 

今回はDCGANというモデルを実装していく
DCGANとは、Deep Convolutional Generatorを用いるGANの一種だ。もう1つの特徴として、活性化関数にLeakyReLUを用いることが挙げられる。

今回は

In [1]:
# ライブラリのインポート

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import tqdm

In [2]:
# Writer will output to ./runs/ directory by default
writer = SummaryWriter('./param/05_3/')

In [3]:
img_data = ImageFolder(
    "/home/t4t5u0/Documents/2020_fun_ai_docs/dataset/05/img",
    transform=transforms.Compose([
        transforms.Resize(80),
        transforms.CenterCrop(64),
        transforms.ToTensor()  
    ])
)

batch_size = 128
img_loader = DataLoader(img_data, 
                        batch_size=batch_size, 
                        shuffle=True,
                        num_workers=16)

In [4]:
nz = 100 # 潜在特徴量ベクトルの大きさ
ngf =32 # Generator を介して伝播されるfeature map の深度

class Geneartor(nn.Module):
    def __init__(self):
        super(Geneartor, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
           # nn.ReLU(inplace=True),
            nn.LeakyReLU(0.01,inplace=True),
            
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
           # nn.ReLU(inplace=True),
            nn.LeakyReLU(0.01,inplace=True),
            
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
           # nn.ReLU(inplace=True),
            nn.LeakyReLU(0.01,inplace=True),
            
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            # nn.ReLU(inplace=True),
            nn.LeakyReLU(0.01,inplace=True),
            
            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, x):
        return self.main(x)

![fig1](./fig/05/dcgan_generator.png)

In [5]:
ndf = 32

class Discremenator(nn.Module):
    def __init__(self):
        super(Discremenator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d( ndf*8, 1, 4, 1, 0, bias=False),
            #nn.Sigmoid()
        )
        
    def forward(self, x):
        out = self.main(x)
        return out.squeeze()

In [6]:
#訓練関数を作成する
d = Discremenator().to('cuda:0')
g = Geneartor().to('cuda:0')

#Adamのパラメータは原著参考
opt_d = optim.Adam(d.parameters(), lr=0.0002, betas=(0.5, 0.999))

opt_g = optim.Adam(g.parameters(), lr= 0.0002, betas=(0.5, 0.999))

#交差エントロピー誤差を計算するための補助変数
ones = torch.ones(batch_size).to('cuda:0')
zeros = torch.zeros(batch_size).to('cuda:0')
#loss_fnc = nn.BCEWithLogitsLoss()
loss_fnc = nn.MSELoss()

#モニタリング用の変数 z
fixed_z = torch.randn(batch_size, nz, 1, 1).to('cuda:0')

In [7]:
from statistics import mean

def train_dcgan(g, d, opt_g, opt_d, loader, writer):
    # 生成モデル
    log_loss_g = []
    log_loss_d = []
    for real_img, _ in tqdm.tqdm(loader):
        batch_len = len(real_img)
        
        real_img = real_img.to('cuda:0')
        
        z = torch.randn(batch_len, nz, 1, 1).to('cuda:0')
        fake_img = g(z)
        
        fake_img_tensor = fake_img.detach()
        
        out = d(fake_img)
        loss_g = loss_fnc(out, ones[:batch_len])
        log_loss_g.append(loss_g.item())
        
        # 勾配をクリアしてから微分とパラメータ更新を行う
        d.zero_grad()
        g.zero_grad()
        loss_g.backward()
        opt_g.step()
        
        # 実際の画像に対する識別モデルの評価関数を計算
        real_out = d(real_img)
        loss_d_real  = loss_fnc(real_out, ones[:batch_len])
        
        # PyTorchでは同じTensorを含んだ計算グラフに対して
        # 2回backwardを行うことができないから保存してあった
        # Tensorを使用して無駄な研鑽を省く
        fake_img = fake_img_tensor
        
        # 偽画像に対する識別モデルの評価関数の計算
        fake_out = d(fake_img_tensor)
        loss_d_fake = loss_fnc(fake_out, zeros[:batch_len])
        
        # 真偽の評価関数の合計値
        loss_d = loss_d_real + loss_d_fake
        log_loss_d.append(loss_d.item())
        
        # 識別モデルの微分計算とパラメータ更新
        d.zero_grad()
        g.zero_grad()
        loss_d.backward()
        opt_d.step()
        
        if writer is not None:
            writer.add_scalars('loss', {
                'D':log_loss_d[-1], 'G':log_loss_g[-1]
                }, epoch)
        
    return mean(log_loss_g), mean(log_loss_d)

In [ ]:
for epoch in range(3000):
    train_dcgan(g, d, opt_g, opt_d, img_loader, writer)
    
    if epoch%10 == 0:
        # パラメータの保存
        torch.save(
            g.state_dict(),
            "./param/05_3/g_{:03d}.prm".format(epoch),
            pickle_protocol = 4
        )
        torch.save(
            d.state_dict(),
            "./param/05_3/d_{:03d}.prm".format(epoch),
            pickle_protocol = 4
        )
        generated_img = g(fixed_z)
        generated_img = g(fixed_z)
        save_image(generated_img,
                  "./param/05_3/{:04d}.jpg".format(epoch)
        )
        

  3%|▎         | 45/1583 [00:01<00:54, 28.32it/s]